# Optimization & Training

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim      # Optimizer
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1   # shuffle 시 동일하게 섞기
# DataLoader() 에서 shuffle=True 을 주어 데이터를 섞을텐데
# 매번 다르게 섞는게 아니라 동일하게 (seed) 섞을수 있도록 함.

batch_size = 64
test_batch_size = 64

no_cuda = False

In [4]:
use_cuda = not no_cuda and torch.cuda.is_available()   # cuda 사용여부
device = torch.device('cuda' if use_cuda else 'cpu')
device

device(type='cpu')

In [6]:
save_dir = r'D:\DevRoot\DataSet\torch\dataset'

# Preprocess

In [7]:
torch.manual_seed(seed)   # shuffle 시 매번 동일하게 shuffle됨 

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        save_dir, train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
    ),
    batch_size = batch_size,
    shuffle = True
)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        save_dir, train=False, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
    ),
    batch_size = test_batch_size,
    shuffle = True
)


# Model

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Optimization 

In [9]:
model = Net().to(device)  # model 불러와서 device 에 담아줌.

In [11]:
# 학습 paramete 확인하는 방법
list(model.parameters())

[Parameter containing:
 tensor([[[[ 0.1031, -0.0883, -0.0388,  0.0939, -0.1883],
           [ 0.1199, -0.0411,  0.1017,  0.0278, -0.0245],
           [ 0.0555,  0.0099,  0.0730, -0.0779, -0.0146],
           [-0.0180,  0.0290, -0.0008,  0.1748,  0.0622],
           [-0.0745, -0.1208, -0.0335, -0.0863, -0.0641]]],
 
 
         [[[ 0.0096,  0.1192,  0.1087, -0.1955,  0.1240],
           [ 0.0559,  0.1897,  0.1320, -0.1822, -0.1902],
           [-0.0965,  0.1756, -0.0333,  0.0856, -0.0929],
           [ 0.1962, -0.0846,  0.1500,  0.0024, -0.1054],
           [ 0.1028, -0.1062,  0.0588, -0.0578, -0.0219]]],
 
 
         [[[-0.1923, -0.0954,  0.1085, -0.0486,  0.1992],
           [ 0.1603, -0.0094, -0.1335,  0.1218,  0.0621],
           [-0.1293,  0.1299,  0.1214,  0.1774, -0.1121],
           [-0.0329, -0.0039,  0.0292, -0.1518, -0.1419],
           [ 0.1088, -0.0469,  0.0977,  0.0114,  0.0657]]],
 
 
         [[[ 0.0440,  0.0727,  0.0991, -0.1852,  0.1007],
           [-0.1406, -0.1509,  

In [13]:
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


In [ ]:
# torch.Size([20, 1, 5, 5])  <-- Conv 의 weight size
# torch.Size([20])           <-- bias
# torch.Size([50, 20, 5, 5]) <-- Conv 의 weight size
# torch.Size([50])           <-- bias
# torch.Size([500, 800])     <-- fully connected..
# torch.Size([500])
# torch.Size([10, 500])
# torch.Size([10])

# keras 에선 model summary 기능이 있었다
# ↑ PyTorch 는 summary 기능이 없기 때문에 이렇게 함 들여다 볼수 있다.

In [14]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
            # SGD : Stochastic Gradient Descent
            # 1. model 의 parameter 들 입력
            # 2. lr (learning rate)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

**momentum** 이란?

![](https://media.vlpt.us/images/reversesky/post/780406cf-3482-4e67-b09a-a955d5be1d1c/image.png)

loss의 미분값이 파라미터의 값이 0에 존재한다고 생각해보자. 위의 사진을 보면, 우리가 구하려는 전역 최소값에 가기전, 지역 최소값에 도달하면 미분값이 0이 되면서 더 이상 움직이지 않는다. 그래서 sgd에 약간의 변형을 준 것이 SGD+Momentum이라는 개념이다.


sgd에다가 이전의 이동값을 고려하도록 설계하여 momentum. 즉 관성을 주었다. 혹여 지역 최소값에 도달하더라도 앞으로 나아가서 지역 최소값을 탈출할 수 있도록 설정해준다.


※참고로 Adam 은 sgd + Momentum + RMSProp 을 같이 사용함 (일반적으로 추천)


# Before Training

- 학습하기 전에 Model이 Train할 수 있도록 Train Mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, DropOut과 추후에 배우게 될 Batch Normalization과 같이 parameter를 가진 Layer들도 학습하기 위해 준비 

In [15]:
model.train()   # 학습하기 전에 Train mode 로 전환해야 함

# train mode 했다가, evaluation mode 했다가,  다시 학습할때는 train mode 로 돌아와야 한다.

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [16]:
data, target = next(iter(train_loader))  # 첫 batch 만 꺼내와 보기 (현재 batch size 는 64)

In [17]:
# 확인
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 Batch 데이터를 device 에 compile 
    - device 는 'cpu' 또는 'gpu'..

In [19]:
data, target = data.to(device), target.to(device)

In [20]:
data.shape, target.shape  # shape 가 바뀌는 건 없다

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [22]:
# 앞에서 Optimizer 를 설정해 주었다.
# 학습하기 전에 Optimizer 를 clear 해 주어야 한다
# zero_grad()  -> 새로운 최적값을 찾기 위해 

In [23]:
optimizer.zero_grad()  # Optimizer clear

- 준비한 데이터를 model 에 input 으로 넣어 output 얻음

In [24]:
output = model(data)

In [26]:
output.shape

torch.Size([64, 10])

In [27]:
# output 이 나왔으니
# 얼마나 틀렸는지 (Loss) 를 봐야 한다

# output 을 target 과 비교하여 얼마나 틀렸는지를 loss function  에 넣어본다.

- Model 에서 예측한 결과를 Loss Function 에 넣음
- 이번 예제에선 Negative Log-Likelihood Loss 라는 Loss Function 사용

In [31]:
loss = F.nll_loss(output, target)

loss

tensor(2.2922, grad_fn=<NllLossBackward>)

- Back Propagation 을 통해 gradients 를 계산

In [32]:
loss.backward()   # 기울기(gradient) 계산



RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [33]:
# 기울기가 계산되었다고 끝이 아니다 -> Optimizer 에 업데이트 해주어야 한다. (Parameter Update)

optimizer.step()

이상이 '학습' 의 "1 스텝"입니다
- train 모드 변환
- 데이터 넣어주고
- 기울기 clear
- model 에 데이터 넣고
- loss 계산하고
- back propagation 하여 gradient 계산하고
- parameter 업데이트


# Training
위의 최적화 과정을 반복하여 학습 시작

In [34]:
# Hypter paramete 설정
epochs = 1
log_interval = 100  # 로그를 확인하기 위해 몇 스텝마다 볼지 지정

In [35]:
for epoch in range(1, epochs + 1):
    
    # 1. train 모드 변환
    model.train()
    
    # for 한번 할때마다 한번 학습 이루어짐
    # 2. 데이터 넣어주기
    
    for batch_idx, (data, target) in enumerate(train_loader): # 매 iteration 마다 batch index 받아온다.
        data, target = data.to(device), target.to(device)    # 데이터를 device 에 compile
        # 3. 기울기 clear
        optimizer.zero_grad()
        # 4. model 에 데이터 넣기
        output = model(data)
        # 5. loss 계산
        loss = F.nll_loss(output, target)
        # 6. back propagation 하여 gradient 계산
        loss.backward()
        # 7. parameter 업데이트
        optimizer.step()
        
        if batch_idx % log_interval == 0:  # 중간중간에 로그 확인
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader),
                loss.item()
            ))
    

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295366
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.220284
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.136016
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.953834
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.498464
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.254101
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.842483
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.706224
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.586595
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.612400
